In [ ]:
# change kernel path
import os
os.chdir('/data1/wangding/project/undefined/lightning-hydra-template/')
os.curdir

In [1]:
import os
import time
from argparse import ArgumentParser
from pprint import pprint
from tqdm import tqdm

import torch
from torch import nn

from torchmetrics.functional import psnr, ssim, mean_squared_error

from src.datamodules.snow_dm import PairedSnowDataModule
from src.model.ablation import BranchBD

device = 4
batch_size = 1
weight = 'logs/all_UNetResRD_bs32_ablation/desnow-epoch=0261-valid_loss=0.074254.ckpt'

# model
model = UNetResRD.load_from_checkpoint(weight)
model.cuda(device).freeze()

l1_loss = nn.L1Loss()

for data_dir in ['S','M','L']:
    # data module
    dm = SnowDataModule(
        batch_size=batch_size,
        data_dir=data_dir,
    )
    dm.setup('test')
    test_dt_len = len(dm.test_dt)

    # test metric and pic
    mse_list = []
    l1_list = []
    psnr_list = []
    ssim_list = []

    with torch.no_grad():
        for i in tqdm(range(test_dt_len)):
            s, g, m = dm.test_dt[i]
            s, g, m = s.cuda(device).unsqueeze(0), g.cuda(device).unsqueeze(0), m.cuda(device).unsqueeze(0)

            d = model(s)
            d = torch.clamp(d, 0, 1)

            mse_list.append(mean_squared_error(d, g).item())
            l1_list.append(l1_loss(d, g).item())
            psnr_list.append(psnr(d, g).item())
            ssim_list.append(ssim(d, g).item())

    mean_mse = sum(mse_list) / test_dt_len
    mean_l1 = sum(l1_list) / test_dt_len
    mean_psnr = sum(psnr_list) / test_dt_len
    mean_ssim = sum(ssim_list) / test_dt_len

    result = {
        'mean_mse': mean_mse,
        'mean_l1': mean_l1,
        'mean_psnr': mean_psnr,
        'mean_ssim': mean_ssim,
    }
    
    print('test dataset: {}'.format(data_dir))
    pprint(result)

NameError: name 'UNetResRD' is not defined